# Pandas数据导入存储与格式
- 文本格式
    - Parsing functions in pandas
    - pd.read_csv('filepath')
    - pd.read_table('filepath', sep=',')
    - pd.read_csv('filepath', header=None):不带文件头
    - pd.read_csv('filepath', names=['col1', 'col2', 'col3']):指定文件头（列名）
    - 指定names和index
        - names = ['col1', 'col2', 'col3']
        - pd.read_csv('filepath', names=names, index_col='col3')
    - 指定层次化index
        - pd.read_csv('filepath', index_col=['col1', 'col2'])
    - 使用正则表达式作为分隔符
        - pd.read_table('filepath', sep='\s+')
    - 忽略某些行：skiprows
        - pd.read_csv('filepath', skiprows=[0, 2, 3]):忽略第0，2，3行
    - 缺失值:NA, -1.#IND, and NULL
        - 检验：pd.isnull(dataframeObj)
        - 指定特定形式的缺失值
            - sentinels = {'col1': ['foo', 'NA'], 'col3': ['two']}
            - pd.read_csv('filepath', na_values=sentinels)
            - 即NA,foo,two都被定义为缺失值
    - read_csv / read_table function arguments
- 读入部分数据
    - pd.options.display.max_rows
    - 设置浏览的行数
        - pd.read_csv('filepath', nrows=5)
    - iterate方式

```py
chunker = pd.read_csv('data/dataload/ex6.csv', chunksize=1000)

tot = pd.Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)
```

- 导出数据到文本
    - data.to_csv('filepath')
    - 指定分割符
        - import sys
        - data.to_csv(sys.stdout, sep='|')
    - 缺失值显式地“展现”
        - data.to_csv(sys.stdout, na_rep='NULL')
    - 关闭index和header
        - data.to_csv(sys.stdout, index=False, header=False)
    - 指定列
        - data.to_csv(sys.stdout, index=False, columns=['col1', 'col2', 'col3'])
- 手动操作数据

```py
import csv
f = open('data/dataload/ex7.csv')

reader = csv.reader(f)
for line in reader:
     print(line)
```

    - csv.Dialect:设置读取csv的配置信息
    - CSV dialect options
- JSON数据
    - jsonStr = json.loads(jsonObj)
    - jsonObj = json.dumps(jsonStr)
    - 转成dataframe
        - pd.DataFrame(jsonStr['json_key'], columns=['json_key_1', 'json_key_2'])
    - data = pd.read_json('jsonfile')
        - data.to_json()
        - data.to_json(orient='records') 
- XML 与 HTML
    - pd.read_html('htmlfilepath')
    - objectify.parse(open('xmlfilepath'))
- 二进制文件格式
    - dataframeObj.to_pickle('picklepath')
    - pd.read_pickle('picklepath')
- 使用HDF5格式
    - store = pd.HDFStore('h5filepath')
    - store['key1'] = dataframeObj
    - store['key1_1'] = dataframeObj['col1']
    - store.put('key2', frame, format='table')
    - dataframeObj.to_hdf('h5filepath', 'key3', format='table')
    - pd.read_hdf('h5filepath', 'key3', where=['index < 5'])
- 读取Excel文件
    - 读
        - xlsx = pd.ExcelFile('xlsxfilepath')
        - pd.read_excel(xlsx, 'Sheet1')
    - 写
        - writer = pd.ExcelWriter('xlsxfilepath')
        - frame.to_excel(writer, 'Sheet1')
        - writer.save()
- 操作Web API
    - import requests
    - url = 'https://api.github.com/repos/pandas-dev/pandas/issues'
    - resp = requests.get(url)
    - data = resp.json()
    - pd.DataFrame(data, columns=['number', 'title', 'labels', 'state'])
- 数据库操作

In [1]:
# coding:utf-8
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
%pwd

u'/Users/zhangjun/Documents/machine-learning-notes/data-processing'

## 文本格式
Table 6-1. Parsing functions in pandas

Function | Description
---------|------------
read_csv | Load delimited data from a file, URL, or file-like object. Use comma as default delimiter
read_table | Load delimited data from a file, URL, or file-like object. Use tab ('\t') as default delimiter
read_fwf | Read data in fixed-width column format (that is, no delimiters)
read_clipboard | Version of read_table that reads data from the clipboard. Useful for converting tables from web pages
read_excel | Read tabular data from an Excel XLS or XLSX file
read_hdf | Read pandas data from an HDF5 file
read_html | Read all tables found in the given HTML document
read_json | Read data from a JSON (JavaScript Object Notation) string representation
read_msgpack | Read pandas data encoded using the MessagePack binary format
read_pickle | Read an arbitrary object stored in Python pickle format
read_sas | Read a SAS dataset stored in one of the SAS system’s custom storage formats
read_sql | Read the results of a SQL query (using SQLAlchemy) as a pandas DataFrame.
read_stata | Read a dataset from Stata file format
read_feather | Read the Feather binary file format

The optional arguments for these functions may fall into a few categories:

- Indexing: can treat one or more columns as the returned DataFrame, and whether to get column names from the file, the user, or not at all.
- Type inference and data conversion: this includes the user-defined value conversions and custom list of missing value markers.
- Datetime parsing: includes combining capability, including combining date and time information spread over multiple columns into a single column in the result.
- Iterating: support for iterating over chunks of very large files.
- Unclean data issues: skipping rows or a footer, comments, or other minor things like numeric data with thousands separated by commas.

In [2]:
!cat data/dataload/ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

Since this is comma-delimited, we can use `read_csv` to read it into a DataFrame:

In [3]:
df = pd.read_csv('data/dataload/ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


We could also have used `read_table` and specifying the delimiter:

In [4]:
pd.read_table('data/dataload/ex1.csv', sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


A file will not always have a header row. Consider this file:

In [5]:
!cat data/dataload/ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

You can allow pandas to assign default column names, or you can specify names yourself:

In [6]:
pd.read_csv('data/dataload/ex2.csv', header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [7]:
pd.read_csv('data/dataload/ex2.csv', names=['a', 'b', 'c', 'd', 'message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Suppose you wanted the message column to be the index of the returned DataFrame. You can either indicate you want the column at index 4 or named 'message' using the `index_col` argument:

In [8]:
names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv('data/dataload/ex2.csv', names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In the event that you want to form a `hierarchical index` from multiple columns, pass a list of column numbers or names:

In [9]:
!cat data/dataload/csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [10]:
parsed = pd.read_csv('data/dataload/csv_mindex.csv', index_col=['key1', 'key2'])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In some cases, a table might not have a fixed delimiter, using whitespace or some other pattern to separate fields. In these cases, you can pass a `regular expression` as a delimiter for `read_table`. Consider a text file that looks like this:

In [12]:
list(open('data/dataload/ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

In [13]:
result = pd.read_table('data/dataload/ex3.txt', sep='\s+')
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


For example, you can skip the first, third, and fourth rows of a file with `skiprows`:

In [14]:
!cat data/dataload/ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [15]:
pd.read_csv('data/dataload/ex4.csv', skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Missing data is usually either not present (empty string) or marked by some sentinel value. By default, pandas uses a set of commonly occurring `sentinels`, such as NA, -1.#IND, and NULL:

In [16]:
!cat data/dataload/ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo

In [17]:
result = pd.read_csv('data/dataload/ex5.csv')
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [18]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


The `na_values` option can take either a list or set of strings to consider missing values:

In [19]:
result = pd.read_csv('data/dataload/ex5.csv', na_values=['NULL'])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


Different NA sentinels can be specified for each column in a dict:

In [20]:
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
pd.read_csv('data/dataload/ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


Table 6-2. Some read_csv / read_table function arguments

Argument | Description
---------|------------
path | String indicating filesystem location, URL, or file-like object
sep or delimiter | Character sequence or regular expression to use to split fields in each row
header | Row number to use as column names. Defaults to 0 (first row), but should be None if there is no header row
index_col | Column numbers or names to use as the row index in the result. Can be a single name/number or a list of them for a hierarchical index
names | List of column names for result, combine with header=None
skiprows | Number of rows at beginning of file to ignore or list of row numbers (starting from 0) to skip
na_values | Sequence of values to replace with NA
comment | Character or characters to split comments off the end of lines
parse_dates | Attempt to parse data to datetime; False by default. If True, will attempt to parse all columns. Otherwise can specify a list of column numbers or name to parse. If element of list is tuple or list, will combine multiple columns together and parse to date (for example if date/time split across two columns)
keep_date_col | If joining columns to parse date, keep the joined columns. Default False
converters | Dict containing column number of name mapping to functions. For example {'foo': f} would apply the function f to all values in the 'foo' column
dayfirst | When parsing potentially ambiguous dates, treat as international format (e.g. 7/6/2012 -> June 7, 2012). Default False
date_parser | Function to use to parse dates
nrows | Number of rows to read from beginning of file
iterator | Return a TextParser object for reading file piecemeal
chunksize | For iteration, size of file chunks
skip_footer | Number of lines to ignore at end of file
verbose | Print various parser output information, like the number of missing values placed in non-numeric columns
encoding | Text encoding for unicode. For example 'utf-8' for UTF-8 encoded text
squeeze | If the parsed data only contains one column return a Series
thousands | Separator for thousands, e.g. ',' or '.'

## 读入部分数据

In [21]:
pd.options.display.max_rows = 10  # Make display more compact
result = pd.read_csv('data/dataload/ex6.csv')
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


If you want to only read out a small number of rows (avoiding reading the entire file), specify that with `nrows`:

In [22]:
pd.read_csv('data/dataload/ex6.csv', nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


The `TextParser` object returned by read_csv allows you to iterate over the parts of the file according to the `chunksize`.

In [23]:
chunker = pd.read_csv('data/dataload/ex6.csv', chunksize=1000)

tot = pd.Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)

tot = tot.sort_values(ascending=False)
tot[:10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

`TextParser` is also equipped with a `get_chunk` method which enables you to read pieces of an arbitrary size.

## 导出数据到文本

In [24]:
data = pd.read_csv('data/dataload/ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


Using DataFrame’s `to_csv` method, we can write the data out to a comma-separated file:

In [25]:
data.to_csv('data/out/out.csv')

In [26]:
!cat data/out/out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


Other delimiters can be used, of course (writing to `sys.stdout` so it prints the text result to the console):

In [27]:
import sys
data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


Missing values appear as empty strings in the output. You might want to denote them by some other `sentinel` value:

In [28]:
data.to_csv(sys.stdout, na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


With no other options specified, both the row and column labels are written. Both of these can be disabled:

In [29]:
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


You can also write only a subset of the columns, and in an order of your choosing:

In [30]:
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


Series also has a `to_csv` method:

In [31]:
dates = pd.date_range('1/1/2000', periods=7)
dates

DatetimeIndex(['2000-01-01', '2000-01-02', '2000-01-03', '2000-01-04',
               '2000-01-05', '2000-01-06', '2000-01-07'],
              dtype='datetime64[ns]', freq='D')

In [32]:
ts = pd.Series(np.arange(7), index=dates)
ts.to_csv('data/out/tseries.csv')
!cat data/out/tseries.csv

2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
2000-01-06,5
2000-01-07,6


## 手动操作数据

In [34]:
!cat data/dataload/ex7.csv

"a","b","c"
"1","2","3"
"1","2","3","4"


In [35]:
import csv
f = open('data/dataload/ex7.csv')

reader = csv.reader(f)
for line in reader:
     print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3', '4']


In [36]:
lines = list(csv.reader(open('data/dataload/ex7.csv')))
header, values = lines[0], lines[1:]
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

Defining a new format with a different delimiter, string quoting convention, or line terminator is done by defining a simple subclass of `csv.Dialect`:

In [37]:
class my_dialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL

reader = csv.reader(f, dialect=my_dialect)

Table 6-3. CSV dialect options

Argument | Description
---------|------------
delimiter | One-character string to separate fields. Defaults to ','.
lineterminator | Line terminator for writing, defaults to '\r\n'. Reader ignores this and recognizes cross-platform line terminators.
quotechar | Quote character for fields with special characters (like a delimiter). Default is '"'.
quoting | Quoting convention. Options include csv.QUOTE_ALL (quote all fields), csv.QUOTE_MINIMAL (only fields with special characters like the delimiter), csv.QUOTE_NONNUMERIC, and csv.QUOTE_NONE (no quoting). See Python’s documentation for full details. Defaults to QUOTE_MINIMAL.
skipinitialspace | Ignore whitespace after each delimiter. Default False.
doublequote | How to handle quoting character inside a field. If True, it is doubled. See online documentation for full detail and behavior.
escapechar | String to escape the delimiter if quoting is set to csv.QUOTE_NONE. Disabled by default

To write delimited files manually, you can use `csv.writer`.

In [38]:
with open('data/out/mydata.csv', 'w') as f:
    writer = csv.writer(f, dialect=my_dialect)
    writer.writerow(('one', 'two', 'three'))
    writer.writerow(('1', '2', '3'))
    writer.writerow(('4', '5', '6'))
    writer.writerow(('7', '8', '9'))

In [39]:
!cat data/out/mydata.csv

one;two;three
1;2;3
4;5;6
7;8;9


## JSON数据

In [40]:
import json

obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 29, "pets": ["Zeus", "Zuko"]},
              {"name": "Katie", "age": 38,
               "pets": ["Sixes", "Stache", "Cisco"]}]
}
"""

result = json.loads(obj)
result

{u'name': u'Wes',
 u'pet': None,
 u'places_lived': [u'United States', u'Spain', u'Germany'],
 u'siblings': [{u'age': 29, u'name': u'Scott', u'pets': [u'Zeus', u'Zuko']},
  {u'age': 38, u'name': u'Katie', u'pets': [u'Sixes', u'Stache', u'Cisco']}]}

In [41]:
asjson = json.dumps(result)

Conveniently, you can pass a list of JSON objects to the DataFrame constructor and select a subset of the data fields:

In [42]:
siblings = pd.DataFrame(result['siblings'], columns=['name', 'age'])
siblings

,name,age
0,Scott,29
1,Katie,38


The default options for `pandas.read_json` assume that each object in the JSON array is a row in the table:

In [44]:
data = pd.read_json('data/dataload/example.json')
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


If you need to export data from pandas to JSON, one way is to use the `to_json` methods on Series and DataFrame:

In [45]:
print(data.to_json())

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}


In [46]:
print(data.to_json(orient='records'))

[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]


## XML 与 HTML

First, you must install some additional libraries used by `read_html`:

```
conda install lxml
pip install beautifulsoup4 html5lib
```

The `pandas.read_html` function has a number of options, but by default it searches for and attempts to parse all tabular data contained within `<table>` tags.

In [48]:
tables = pd.read_html('data/dataload/fdic_failed_bank_list.html')
len(tables)

1

In [49]:
failures = tables[0]
failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


In [50]:
close_timestamps = pd.to_datetime(failures['Closing Date'])
close_timestamps.dt.year.value_counts()

2010    157
2009    140
2011     92
2012     51
2008     25
       ... 
2004      4
2001      4
2007      3
2003      3
2000      2
Name: Closing Date, dtype: int64

### PARSING XML WITH LXML.OBJECTIFY
The New York Metropolitan Transportation Authority (MTA) publishes a number of data series about its bus and train services (http://www.mta.info/developers/download.html). Here we’ll look at the performance data which is contained in a set of XML files. Each train or bus service has a different file (like Performance_MNR.xml for the Metro-North Railroad) containing monthly data as a series of XML records that look like this:

```xml
<INDICATOR>
  <INDICATOR_SEQ>373889</INDICATOR_SEQ>
  <PARENT_SEQ></PARENT_SEQ>
  <AGENCY_NAME>Metro-North Railroad</AGENCY_NAME>
  <INDICATOR_NAME>Escalator Availability</INDICATOR_NAME>
  <DESCRIPTION>Percent of the time that escalators are operational
  systemwide. The availability rate is based on physical observations performed
  the morning of regular business days only. This is a new indicator the agency
  began reporting in 2009.</DESCRIPTION>
  <PERIOD_YEAR>2011</PERIOD_YEAR>
  <PERIOD_MONTH>12</PERIOD_MONTH>
  <CATEGORY>Service Indicators</CATEGORY>
  <FREQUENCY>M</FREQUENCY>
  <DESIRED_CHANGE>U</DESIRED_CHANGE>
  <INDICATOR_UNIT>%</INDICATOR_UNIT>
  <DECIMAL_PLACES>1</DECIMAL_PLACES>
  <YTD_TARGET>97.00</YTD_TARGET>
  <YTD_ACTUAL></YTD_ACTUAL>
  <MONTHLY_TARGET>97.00</MONTHLY_TARGET>
  <MONTHLY_ACTUAL></MONTHLY_ACTUAL>
</INDICATOR>
```

Using `lxml.objectify`, we parse the file and get a reference to the root node of the XML file with `getroot`:


In [51]:
from lxml import objectify

path = 'data/dataload/mta_perf/Performance_MNR.xml'
parsed = objectify.parse(open(path))
root = parsed.getroot()

`root.INDICATOR` return a generator yielding each <INDICATOR> XML element. For each record, we can populate a dict of tag names (like YTD_ACTUAL) to data values (excluding a few tags):

In [53]:
data = []

skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ',
               'DESIRED_CHANGE', 'DECIMAL_PLACES']

for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

Lastly, convert this list of dicts into a DataFrame:

In [54]:
perf = pd.DataFrame(data)
perf.head()

,AGENCY_NAME,CATEGORY,DESCRIPTION,FREQUENCY,INDICATOR_NAME,INDICATOR_UNIT,MONTHLY_ACTUAL,MONTHLY_TARGET,PERIOD_MONTH,PERIOD_YEAR,YTD_ACTUAL,YTD_TARGET
0,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,1,2008,96.9,95
1,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95,95,2,2008,96,95
2,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,3,2008,96.3,95
3,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,98.3,95,4,2008,96.8,95
4,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95.8,95,5,2008,96.6,95


## 二进制文件格式
pandas objects all have a `to_pickle` method which writes the data to disk in pickle format:

In [55]:
frame = pd.read_csv('data/dataload/ex1.csv')
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [56]:
frame.to_pickle('data/out/frame_pickle')

Any “pickled” object stored in a file can be read by using the built-in pickle directly, or even more conveniently using pandas.`read_pickle`:

In [57]:
pd.read_pickle('data/out/frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


## 使用HDF5格式
HDF5 is a well-regarded file format intended for storing large quantities of scientific array data.

The `HDFStore` class works like a dict and handles the low-level details:

In [58]:
frame = pd.DataFrame({'a': np.random.randn(100)})
store = pd.HDFStore('data/out/mydata.h5')
store['obj1'] = frame
store['obj1_col'] = frame['a']
store

<class 'pandas.io.pytables.HDFStore'>
File path: data/out/mydata.h5
/obj1                frame        (shape->[100,1])
/obj1_col            series       (shape->[100])  

In [59]:
store['obj1']

,a
0,0.429688
1,-1.157171
2,0.917111
3,0.808405
4,0.231996
...,...
95,-1.000404
96,-2.345141
97,0.706472
98,0.296808


HDFStore supports two storage schemas, `'fixed'` and `'table'`. The latter is generally slower, but it supports query operations using a special syntax

In [60]:
store.put('obj2', frame, format='table')

The pandas.`read_hdf` function gives you a shortcut to these tools:

In [61]:
frame.to_hdf('data/out/mydata.h5', 'obj3', format='table')
pd.read_hdf('data/out/mydata.h5', 'obj3', where=['index < 5'])

,a
0,0.429688
1,-1.157171
2,0.917111
3,0.808405
4,0.231996


## 读取Excel文件
pandas also supports reading tabular data stored in Excel 2003 (and higher) files using either the `ExcelFile` class or pandas.`read_excel` function. Internally these tools use the add-on packages `xlrd` and `openpyxl` to read XLS and XLSX files, respectively. You may need to install these manually with pip or conda.

In [62]:
xlsx = pd.ExcelFile('data/dataload/ex1.xlsx')
pd.read_excel(xlsx, 'Sheet1')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [63]:
frame = pd.read_excel('data/dataload/ex1.xlsx', 'Sheet1')
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


To write pandas data to Excel format, you must first create an `ExcelWriter`, then write data to it using pandas objects’ `to_excel` method:

In [64]:
writer = pd.ExcelWriter('data/out/ex2.xlsx')
frame.to_excel(writer, 'Sheet1')
writer.save()

## 操作Web API

In [66]:
import requests

url = 'https://api.github.com/repos/pandas-dev/pandas/issues'
resp = requests.get(url)
resp

<Response [200]>

In [67]:
data = resp.json()
data[0]['title']

u'DOC: correct grammar in unicode section'

In [68]:
issues = pd.DataFrame(data, columns=['number', 'title', 'labels', 'state'])
issues

,number,title,labels,state
0,17678,DOC: correct grammar in unicode section,[],open
1,17677,Melt enhance,[],open
2,17676,Simultaneously melt multiple columns,[{u'url': u'https://api.github.com/repos/panda...,open
3,17675,DOC: Add examples for MultiIndex.get_locs + cl...,[{u'url': u'https://api.github.com/repos/panda...,open
4,17674,as_block depr warning on tab completion,[{u'url': u'https://api.github.com/repos/panda...,open
...,...,...,...,...
25,17628,BUG: coerce pd.wide_to_long suffixes to ints,[{u'url': u'https://api.github.com/repos/panda...,open
26,17627,wide_to_long does not convert integer suffixes...,[{u'url': u'https://api.github.com/repos/panda...,open
27,17621,BUG: Fix unexpected sort in groupby,[{u'url': u'https://api.github.com/repos/panda...,open
28,17618,BUG: roundtrip fidelity for a tz-aware DTI wit...,[{u'url': u'https://api.github.com/repos/panda...,open


## 数据库操作

In [78]:
import sqlite3

query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
 c REAL,        d INTEGER
);"""

con = sqlite3.connect('mydata.sqlite')
con.execute(query)
con.commit()

data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"

con.executemany(stmt, data)
con.commit()

In [79]:
cursor = con.execute('select * from test')
rows = cursor.fetchall()
rows

[(u'Atlanta', u'Georgia', 1.25, 6),
 (u'Tallahassee', u'Florida', 2.6, 3),
 (u'Sacramento', u'California', 1.7, 5)]

In [80]:
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [81]:
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


In [86]:
import sqlalchemy as sqla

db = sqla.create_engine('sqlite:///mydata.sqlite')

In [87]:
pd.read_sql('select * from test', db)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


In [88]:
!rm mydata.sqlite